In [1]:
import json
import tqdm

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
torch.set_default_device('cuda')

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", trust_remote_code=True, torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True, torch_dtype="auto")


Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [5]:
def phi_response(prompt, tokenizer,model):
    prompt_text = prompt["refined_prompt"]+ "\nGenerate a RegEx for this description.\n Answer:"
    inputs = tokenizer(prompt_text, return_tensors="pt")
    x = inputs['input_ids']
    x = x.expand(10, -1)
    generated_token = model.generate(
        x,
        temperature=0.8,
        max_length=128,
        do_sample=True,
    )
    prompt["phi_output"] =[]
    for i in range(10):
        prompt["phi_output"].append({})
        output = generated_token[i].cpu().squeeze()
        prompt["phi_output"][i]["text"] = tokenizer.decode(output).split(prompt_text)[-1].split("\n\n")[0]

    return prompt


In [4]:
with open('./RegexEval_v4.json') as f:
    data = json.loads(f.read())

len(data)

762

In [7]:
new_data = []
for item in tqdm.tqdm(data):

    item = phi_response(item, tokenizer,model)
    print(item)
    new_data.append(item)


  0%|          | 0/762 [00:00<?, ?it/s]


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /var/folders/9d/98myd0lx1rn2j76hx3t96x1r0000gn/T/ipykernel_61882/3148686149.py:4 in       │
│ <module>                                                                                  │
│                                                                                           │
│ [Errno 2] No such file or directory:                                                      │
│ '/var/folders/9d/98myd0lx1rn2j76hx3t96x1r0000gn/T/ipykernel_61882/3148686149.py'          │
│ /var/folders/9d/98myd0lx1rn2j76hx3t96x1r0000gn/T/ipykernel_61882/860171999.py:5 in        │
│ phi_response                                                                              │
│                                                                                           │
│ [Errno 2] No such file or directory:                                                      │
│ '/var/folders/9d/98myd0lx1rn2j76hx3t96x1r0000gn/T/ipykernel_61882/860171999.py'           │
│                                                                                           │
│ /Users/lsiddiqsunny/.pyenv/versions/3.9.4/lib/python3.9/site-packages/torch/autograd/grad │
│ _mode.py:27 in decorate_context                                                           │
│                                                                                           │
│    24 │   │   @functools.wraps(func)                                                      │
│    25 │   │   def decorate_context(*args, **kwargs):                                      │
│    26 │   │   │   with self.clone():                                                      │
│ ❱  27 │   │   │   │   return func(*args, **kwargs)                                        │
│    28 │   │   return cast(F, decorate_context)                                            │
│    29 │                                                                                   │
│    30 │   def _wrap_generator(self, func):                                                │
│                                                                                           │
│ /Users/lsiddiqsunny/.pyenv/versions/3.9.4/lib/python3.9/site-packages/transformers/genera │
│ tion_utils.py:1541 in generate                                                            │
│                                                                                           │
│   1538 │   │   │   )                                                                      │
│   1539 │   │   │                                                                          │
│   1540 │   │   │   # 12. run sample                                                       │
│ ❱ 1541 │   │   │   return self.sample(                                                    │
│   1542 │   │   │   │   input_ids,                                                         │
│   1543 │   │   │   │   logits_processor=logits_processor,                                 │
│   1544 │   │   │   │   logits_warper=logits_warper,                                       │
│                                                                                           │
│ /Users/lsiddiqsunny/.pyenv/versions/3.9.4/lib/python3.9/site-packages/transformers/genera │
│ tion_utils.py:2471 in sample                                                              │
│                                                                                           │
│   2468 │   │   │   model_inputs = self.prepare_inputs_for_generation(input_ids, **model_k │
│   2469 │   │   │                                                                          │
│   2470 │   │   │   # forward pass to get next token                                       │
│ ❱ 2471 │   │   │   outputs = self(                                                        │
│   2472 │   │   │   │   **model_inputs,            

In [ ]:
with open('./T5_Refined_Output.json', 'w') as f:
    json.dump(new_data, f, indent=4)